NOTES: In the code, we tried to formulate and estimate 3 types of discrete choice models: MNL, mixed logit (with 3 parameter specifications), and nested logit. Existing issues include:
1. The parameter for $t_\textit{shift}$ is non-significant no matter how we specify the model, which is detailed in the "Model formulation & parameter estimation" section.
2. Pylogit sometimes results in negative $\sigma$ parameters when estimating a mixed logit model, which is unreasonable.
3. The sociodemographic data, i.e. individual characteristics, are generally not included into our choice model. (However, they would make the model harder to interpret because they would form interaction terms.)

In [1]:
import pandas as pd
import numpy as np
import pylogit as pl
from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')

# Data processing

In [2]:
df = pd.read_csv('CE264_April 21, 2022_17.52.csv', skiprows=[1, 2]).fillna('')
df

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,F-2-1-4.2,F-3-2-4.0,F-3-2-4.1,F-3-2-4.2,F-4-1-4.0,F-4-1-4.1,F-4-1-4.2,F-5-1-4.0,F-5-1-4.1,F-5-1-4.2
0,2022-04-05 18:49:43,2022-04-05 18:53:39,Survey Preview,,100,236,True,2022-04-05 18:53:40,R_24Csf53B38FuyLP,,...,,,,,,,,40.0,60.0,90.0
1,2022-04-05 19:18:10,2022-04-05 19:18:21,Survey Preview,,100,10,True,2022-04-05 19:18:22,R_2Bkxjyfqv8O9OYS,,...,,,,,,,,,,
2,2022-04-05 19:59:31,2022-04-05 19:59:32,Survey Test,,100,1,True,2022-04-05 19:59:32,R_9MtQkovIbk1yUkK,,...,,,,,,,,,,
3,2022-04-05 19:59:32,2022-04-05 19:59:32,Survey Test,,100,0,True,2022-04-05 19:59:32,R_ewX7ZEAcLT0YiAC,,...,,,,,,,,,,
4,2022-04-05 19:59:33,2022-04-05 19:59:33,Survey Test,,100,0,True,2022-04-05 19:59:33,R_0d0xD9mN9LXwSrQ,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,2022-04-14 11:05:59,2022-04-14 11:06:58,IP Address,108.30.233.119,71,58,False,2022-04-21 11:07:37,R_10IsOTECUkWVDGY,,...,,,,,,,,,,
102,2022-04-21 12:20:16,2022-04-21 12:22:55,IP Address,184.23.21.197,100,158,True,2022-04-21 12:22:55,R_1M4GCxvdn1XLNQJ,,...,,,,,,,,,,
103,2022-04-21 12:23:13,2022-04-21 12:24:54,IP Address,184.23.21.197,100,101,True,2022-04-21 12:24:55,R_1jv5sVXzZtN55rO,,...,,,,,,,,,,
104,2022-04-21 12:25:00,2022-04-21 12:26:49,IP Address,184.23.21.197,100,109,True,2022-04-21 12:26:49,R_2Crlz1V5Qyzsxwt,,...,,,,,,,,,,


In [3]:
df = df[(df['Q13']!='') & (df['F-1-1-1']!='') & (df['Q8']!='')]

In [4]:
sample = df[['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q13', 'Q14', 'Q15', 'Q16', 
             'F-1-1-1', 'F-1-1-2', 'F-1-1-3', 'F-1-1-4', 'F-1-2-1', 'F-1-2-2', 'F-1-2-3', 'F-1-2-4',
             'F-2-1-1', 'F-2-1-2', 'F-2-1-3', 'F-2-1-4', 'F-2-2-1', 'F-2-2-2', 'F-2-2-3', 'F-2-2-4',
             'F-3-1-1', 'F-3-1-2', 'F-3-1-3', 'F-3-1-4', 'F-3-2-1', 'F-3-2-2', 'F-3-2-3', 'F-3-2-4',
             'F-4-1-1', 'F-4-1-2', 'F-4-1-3', 'F-4-1-4', 'F-4-2-1', 'F-4-2-2', 'F-4-2-3', 'F-4-2-4']]

col_name = {'Q1': 'is_driver', 'Q2': 'age', 'Q3': 'gender', 'Q4': 'occ', 'Q5': 'income', 'Q6': 'hh_size',
            'Q7': 'have_driven', 'Q8': 'trip_purp', 'Q9': 'start_time', 'Q10': 'trip_freq',
            'Q13': 'choice_1', 'Q14': 'choice_2', 'Q15': 'choice_3', 'Q16': 'choice_4',
            'F-1-1-1': 't_detour_1a', 'F-1-1-2': 't_cong_1a', 'F-1-1-3': 't_shift_1a', 'F-1-1-4': 'cost_1a',
            'F-1-2-1': 't_detour_1b', 'F-1-2-2': 't_cong_1b', 'F-1-2-3': 't_shift_1b', 'F-1-2-4': 'cost_1b',
            'F-2-1-1': 't_detour_2a', 'F-2-1-2': 't_cong_2a', 'F-2-1-3': 't_shift_2a', 'F-2-1-4': 'cost_2a',
            'F-2-2-1': 't_detour_2b', 'F-2-2-2': 't_cong_2b', 'F-2-2-3': 't_shift_2b', 'F-2-2-4': 'cost_2b',
            'F-3-1-1': 't_detour_3a', 'F-3-1-2': 't_cong_3a', 'F-3-1-3': 't_shift_3a', 'F-3-1-4': 'cost_3a',
            'F-3-2-1': 't_detour_3b', 'F-3-2-2': 't_cong_3b', 'F-3-2-3': 't_shift_3b', 'F-3-2-4': 'cost_3b',
            'F-4-1-1': 't_detour_4a', 'F-4-1-2': 't_cong_4a', 'F-4-1-3': 't_shift_4a', 'F-4-1-4': 'cost_4a',
            'F-4-2-1': 't_detour_4b', 'F-4-2-2': 't_cong_4b', 'F-4-2-3': 't_shift_4b', 'F-4-2-4': 'cost_4b'}

sample = sample.rename(columns=col_name).reset_index(drop=True)

sample['is_driver'] = sample['is_driver'].map({'Yes': 1, 'No': 0})
sample['age'] = sample['age'].map({'Below 18\t': 1, '18-25': 2, '26-35': 3, '36-45': 4, '45-60': 5, 'above 60': 6})
sample['gender'] = sample['gender'].map({'Male': 1, 'Female': 2, 'Non-binary / third gender': 3,
                                         'Prefer not to say': 4})
sample['occ'] = sample['occ'].map({'Student at UC Berkeley': 1, 'Faculty at UC Berkeley': 2, 'Non UC Berkeley': 3})
sample['income'] = sample['income'].map({'Less than $50k': 1, ' $50k-150k': 2, 'Greater than $150k': 3})
sample['hh_size'] = sample['hh_size'].map({'1': 1, '2': 2, '3': 3, '4': 4, '>=5': 5})
sample['have_driven'] = sample['have_driven'].map({'Yes': 1, 'No': 0})
sample['trip_purp'] = sample['trip_purp'].map({'Work': 1, 'School': 2, 'Shopping': 3, 'Leisure': 4, 'Other': 5})
sample['start_time'] = [int(t) if t.isdigit() else '' for t in sample['start_time']]
sample['trip_freq'] = sample['trip_freq'].map({'1': 1, '2': 2, '3': 3, '4': 4, '>=5': 5})
for i in range(1, 5):
    sample[f'choice_{i}'] = sample[f'choice_{i}'].map({'Alternative A': 1, 'Alternative B': 2, 'Alternative C': 3})

In [5]:
# If we only analyze drivers' choice
# sample = sample[sample['is_driver']==1].reset_index(drop=True)

In [6]:
sample.insert(0, 'id', [i+1 for i in sample.index])

In [7]:
sample_long = sample.iloc[:, :11]
sample_long = sample_long.loc[sample_long.index.repeat(12)].reset_index(drop=True)

t_d, t_c, t_s, c, choice = [[] for i in range(5)]
for i in sample_long.index:
    y = i//12
    if (i%12)%3 == 0:
        x = int((i%12)/3*8)
        t_d.append(sample.iloc[y, 15+x])
        t_c.append(sample.iloc[y, 16+x])
        t_s.append(sample.iloc[y, 17+x])
        c.append(sample.iloc[y, 18+x])
    elif (i%12)%3 == 1:
        x = int((i%12-1)/3*8)
        t_d.append(sample.iloc[y, 19+x])
        t_c.append(sample.iloc[y, 20+x])
        t_s.append(sample.iloc[y, 21+x])
        c.append(sample.iloc[y, 22+x])
    else:
        t_d.append(0)
        t_c.append(0)
        t_s.append(0)
        c.append(0)

sample_long['t_detour'] = t_d
sample_long['t_cong'] = t_c
sample_long['t_shift'] = t_s
sample_long['cost'] = c

for i in sample.index:
    temp = [0 for a in range(12)]
    for j in range(4):
        temp[3*j+sample.iloc[i, 11+j]-1] = 1
    choice.extend(temp)

sample_long['choice'] = choice
sample_long.insert(1, 'alt', [i%3+1 for i in sample_long.index])
sample_long.insert(2, 'sit', [i//3+1 for i in sample_long.index])

In [8]:
sample_long.head(12)

,id,alt,sit,is_driver,age,gender,occ,income,hh_size,have_driven,trip_purp,start_time,trip_freq,t_detour,t_cong,t_shift,cost,choice
0,1,1,1,1,3,3,3,2,5,1,1,,1,35.0,60.0,30.0,20.0,0
1,1,2,1,1,3,3,3,2,5,1,1,,1,45.0,40.0,60.0,10.0,1
2,1,3,1,1,3,3,3,2,5,1,1,,1,0.0,0.0,0.0,0.0,0
3,1,1,2,1,3,3,3,2,5,1,1,,1,25.0,60.0,15.0,15.0,0
4,1,2,2,1,3,3,3,2,5,1,1,,1,45.0,60.0,45.0,0.0,1
5,1,3,2,1,3,3,3,2,5,1,1,,1,0.0,0.0,0.0,0.0,0
6,1,1,3,1,3,3,3,2,5,1,1,,1,65.0,40.0,45.0,20.0,1
7,1,2,3,1,3,3,3,2,5,1,1,,1,35.0,40.0,60.0,10.0,0
8,1,3,3,1,3,3,3,2,5,1,1,,1,0.0,0.0,0.0,0.0,0
9,1,1,4,1,3,3,3,2,5,1,1,,1,65.0,10.0,45.0,0.0,0


# Model formulation & parameter estimation

We noticed that the parameter for $t_\textit{shift}$ is non-significant. Possible reasons include: a) respondents are less sensitive to this variable compared with other variables; 2) respondents generally ignored this variable when answering the survey. Therefore,
1. We may use dummy variables instead that interact with trip purpose to capture the taste heterogeneity. Note that dummy variables would bring another intercept in our model, as mentioned by Joan for Problem Set 2.
2. We may use different parameters for positive $t_\textit{shift}$ (depart later) and negative $t_\textit{shift}$ (depart earlier).

In general, when we include parameters for both positive and negative time shifts, they would both be non-significant, especially the parameter for positive time shifts.

In [9]:
# sample_long['pos_shift'] = [1 if t>0 and (p==1 or p==2) else 0 for t, p in zip(t_s, sample_long['trip_purp'])]
# sample_long['neg_shift'] = [1 if t<0 and (p==1 or p==2) else 0 for t, p in zip(t_s, sample_long['trip_purp'])]

sample_long['pos_shift'] = [t if t>0 else 0 for t in t_s]
sample_long['neg_shift'] = [-t if t<0 else 0 for t in t_s]

## MNL model

In [10]:
basic_spec = OrderedDict()
basic_names = OrderedDict()

# If there are dummy variables, there should be two intercepts
# basic_spec['intercept'] = [1, 3]
# basic_names['intercept'] = ['ASC (route 1)', 'ASC (canceling)']

# If there are no dummy variables, only alternative 3 (canceling) should have an intercept
basic_spec['intercept'] = [3]
basic_names['intercept'] = ['ASC (canceling)']

basic_spec['t_detour'] = [[1, 2]]
basic_names['t_detour'] = ['t_detour (min)']

basic_spec['t_cong'] = [[1, 2]]
basic_names['t_cong'] = ['t_congestion (min)']

basic_spec['pos_shift'] = [[1, 2]]
basic_names['pos_shift'] = ['t_shift_late (min)']

basic_spec['neg_shift'] = [[1, 2]]
basic_names['neg_shift'] = ['t_shift_early (min)']

basic_spec['cost'] = [[1, 2]]
basic_names['cost'] = ['cost ($)']

In [11]:
mnl = pl.create_choice_model(data=sample_long,
                             alt_id_col='alt',
                             obs_id_col='sit',
                             choice_col='choice',
                             specification=basic_spec,
                             model_type='MNL',
                             names=basic_names)

In [12]:
mnl.fit_mle(np.zeros(6))
mnl.get_statsmodels_summary()

Log-likelihood at zero: -329.5837
Initial Log-likelihood: -329.5837
Estimation Time for Point Estimation: 0.01 seconds.
Final log-likelihood: -304.0276


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      choice   No. Observations:                  300
Model:             Multinomial Logit Model   Df Residuals:                      294
Method:                                MLE   Df Model:                            6
Date:                     Sun, 24 Apr 2022   Pseudo R-squ.:                   0.078
Time:                             20:06:35   Pseudo R-bar-squ.:               0.059
AIC:                               620.055   Log-Likelihood:               -304.028
BIC:                               642.278   LL-Null:                      -329.584
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
ASC (canceling)        -1.8562      0.319     -5.819      0.000      -2.481      -1.231
t_detour (min)         -0.0181      0.005     -3.738      0.000      -0.028      -0.009
t_congestion (min)     -0.0173      0.004     -4.322      0.000      -0.025      -0.009
t_shift_late (min)     -0.0033      0.003     -0.986      0.324      -0.010       0.003
t_shift_early (min)    -0.0004      0.004     -0.118      0.906      -0.008       0.007
cost ($)               -0.0513      0.013     -3.975      0.000      -0.077      -0.026
=======================================================================================
"""

## Mixed logit model

In [13]:
mixed_spec = OrderedDict(basic_spec)
mixed_names = OrderedDict(basic_names)

### With random parameters

In [14]:
mixvar1 = ['t_detour (min)', 't_congestion (min)', 'cost ($)']

In [15]:
mixed1 = pl.create_choice_model(data=sample_long,
                                alt_id_col='alt',
                                obs_id_col='sit',
                                choice_col='choice',
                                specification=mixed_spec,
                                model_type='Mixed Logit',
                                names=mixed_names,
                                mixing_id_col='id',
                                mixing_vars=mixvar1)

In [16]:
mixed1.fit_mle(init_vals=np.zeros(9), num_draws=600, seed=1)
mixed1.get_statsmodels_summary()

Log-likelihood at zero: -329.5837
Initial Log-likelihood: -329.5837
Estimation Time for Point Estimation: 4.97 seconds.
Final log-likelihood: -294.9217


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Mixed Logit Model Regression Results                     
==============================================================================
Dep. Variable:                 choice   No. Observations:                  300
Model:              Mixed Logit Model   Df Residuals:                      291
Method:                           MLE   Df Model:                            9
Date:                Sun, 24 Apr 2022   Pseudo R-squ.:                   0.105
Time:                        20:06:40   Pseudo R-bar-squ.:               0.078
AIC:                          607.843   Log-Likelihood:               -294.922
BIC:                          641.177   LL-Null:                      -329.584
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
ASC (canceling)             -2.4552      0.345     -7.121      0.000      -3.131      -1.779
t_detour (min)              -0.0268      0.008     -3.423      0.001      -0.042      -0.011
t_congestion (min)          -0.0269      0.006     -4.154      0.000      -0.040      -0.014
t_shift_late (min)          -0.0042      0.004     -1.131      0.258      -0.012       0.003
t_shift_early (min)          0.0014      0.005      0.296      0.767      -0.008       0.010
cost ($)                    -0.0633      0.017     -3.666      0.000      -0.097      -0.029
Sigma t_detour (min)        -0.0246      0.008     -3.188      0.001      -0.040      -0.009
Sigma t_congestion (min)     0.0245      0.010      2.548      0.011       0.006       0.043
Sigma cost ($)               0.0440      0.026      1.722      0.085      -0.006       0.094
============================================================================================
"""

### With nesting

In [17]:
mixvar2 = ['ASC (canceling)']

In [18]:
mixed2 = pl.create_choice_model(data=sample_long,
                                alt_id_col='alt',
                                obs_id_col='sit',
                                choice_col='choice',
                                specification=mixed_spec,
                                model_type='Mixed Logit',
                                names=mixed_names,
                                mixing_id_col='id',
                                mixing_vars=mixvar2)

In [19]:
mixed2.fit_mle(init_vals=np.zeros(7), num_draws=600, seed=1)
mixed2.get_statsmodels_summary()

Log-likelihood at zero: -329.5837
Initial Log-likelihood: -329.5837
Estimation Time for Point Estimation: 3.37 seconds.
Final log-likelihood: -287.4659


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Mixed Logit Model Regression Results                     
==============================================================================
Dep. Variable:                 choice   No. Observations:                  300
Model:              Mixed Logit Model   Df Residuals:                      293
Method:                           MLE   Df Model:                            7
Date:                Sun, 24 Apr 2022   Pseudo R-squ.:                   0.128
Time:                        20:06:44   Pseudo R-bar-squ.:               0.107
AIC:                          588.932   Log-Likelihood:               -287.466
BIC:                          614.858   LL-Null:                      -329.584
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
ASC (canceling)          -2.6193      0.392     -6.688      0.000      -3.387      -1.852
t_detour (min)           -0.0251      0.006     -4.268      0.000      -0.037      -0.014
t_congestion (min)       -0.0219      0.004     -5.420      0.000      -0.030      -0.014
t_shift_late (min)       -0.0034      0.004     -0.896      0.370      -0.011       0.004
t_shift_early (min)       0.0007      0.004      0.155      0.877      -0.008       0.009
cost ($)                 -0.0572      0.015     -3.848      0.000      -0.086      -0.028
Sigma ASC (canceling)    -1.7171      0.335     -5.118      0.000      -2.375      -1.060
=========================================================================================
"""

### With alternative-specific variances

This specification does not behave well because the $\sigma$'s for routes 1 and 2 are very non-significant.

In [20]:
mixed_spec['intercept'] = [1, 2, 3]
mixed_names['intercept'] = ['ASC (route 1)', 'ASC (route 2)', 'ASC (canceling)']

mixvar3 = ['ASC (route 1)', 'ASC (route 2)', 'ASC (canceling)']

In [21]:
mixed3 = pl.create_choice_model(data=sample_long,
                                alt_id_col='alt',
                                obs_id_col='sit',
                                choice_col='choice',
                                specification=mixed_spec,
                                model_type='Mixed Logit',
                                names=mixed_names,
                                mixing_id_col='id',
                                mixing_vars=mixvar3)

In [22]:
mixed3.fit_mle(init_vals=np.zeros(11), num_draws=600, constrained_pos=[0, 1], seed=1)
mixed3.get_statsmodels_summary()

Log-likelihood at zero: -329.5837
Initial Log-likelihood: -329.5837
Estimation Time for Point Estimation: 3.92 seconds.
Final log-likelihood: -287.1063


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Mixed Logit Model Regression Results                     
==============================================================================
Dep. Variable:                 choice   No. Observations:                  300
Model:              Mixed Logit Model   Df Residuals:                      289
Method:                           MLE   Df Model:                           11
Date:                Sun, 24 Apr 2022   Pseudo R-squ.:                   0.129
Time:                        20:06:48   Pseudo R-bar-squ.:               0.096
AIC:                          596.213   Log-Likelihood:               -287.106
BIC:                          636.954   LL-Null:                      -329.584
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
ASC (route 1)                  0        nan        nan        nan         nan         nan
ASC (route 2)                  0        nan        nan        nan         nan         nan
ASC (canceling)          -2.6138      0.398     -6.571      0.000      -3.393      -1.834
t_detour (min)           -0.0250      0.006     -4.209      0.000      -0.037      -0.013
t_congestion (min)       -0.0219      0.004     -5.365      0.000      -0.030      -0.014
t_shift_late (min)       -0.0034      0.004     -0.886      0.376      -0.011       0.004
t_shift_early (min)       0.0006      0.005      0.142      0.887      -0.008       0.009
cost ($)                 -0.0571      0.015     -3.702      0.000      -0.087      -0.027
Sigma ASC (route 1)      -0.0250      2.488     -0.010      0.992      -4.901       4.851
Sigma ASC (route 2)      -0.0116      3.047     -0.004      0.997      -5.984       5.961
Sigma ASC (canceling)    -1.7364      0.336     -5.170      0.000      -2.395      -1.078
=========================================================================================
"""

## Nested logit model

In [23]:
nest_membership = OrderedDict()
nest_membership['Nest: canceling'] = [3]
nest_membership['Nest: driving'] = [1, 2]

nested_spec = OrderedDict()
nested_names = OrderedDict()

nested_spec['intercept'] = [3]
nested_names['intercept'] = ['ASC (canceling)']

nested_spec['t_detour'] = [[1, 2]]
nested_names['t_detour'] = ['t_detour (min)']

nested_spec['t_cong'] = [[1, 2]]
nested_names['t_cong'] = ['t_congestion (min)']

nested_spec['pos_shift'] = [[1, 2]]
nested_names['pos_shift'] = ['t_shift_late (min)']

nested_spec['neg_shift'] = [[1, 2]]
nested_names['neg_shift'] = ['t_shift_early (min)']

nested_spec['cost'] = [[1, 2]]
nested_names['cost'] = ['cost ($)']

In [24]:
nested = pl.create_choice_model(data=sample_long,
                                alt_id_col='alt',
                                obs_id_col='sit',
                                choice_col='choice',
                                specification=nested_spec,
                                model_type='Nested Logit',
                                names=nested_names,
                                nest_spec=nest_membership)

In [25]:
init_nests = -1 * np.random.rand(2)
init_coefs = np.zeros(6)
init_vals = np.concatenate((init_nests, init_coefs), axis=0)
nested.fit_mle(init_vals, constrained_pos=[0])
nested.get_statsmodels_summary()

Log-likelihood at zero: -337.5391
Initial Log-likelihood: -343.9431
Estimation Time for Point Estimation: 0.22 seconds.
Final log-likelihood: -299.7564


<class 'statsmodels.iolib.summary.Summary'>
"""
                    Nested Logit Model Regression Results                     
==============================================================================
Dep. Variable:                 choice   No. Observations:                  300
Model:             Nested Logit Model   Df Residuals:                      292
Method:                           MLE   Df Model:                            8
Date:                Sun, 24 Apr 2022   Pseudo R-squ.:                   0.112
Time:                        20:06:48   Pseudo R-bar-squ.:               0.088
AIC:                          615.513   Log-Likelihood:               -299.756
BIC:                          645.143   LL-Null:                      -337.539
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Nest: canceling        -0.8107        nan        nan        nan         nan         nan
Nest: driving          -0.9443      1.012     -0.933      0.351      -2.928       1.040
ASC (canceling)        -1.2873      0.334     -3.851      0.000      -1.942      -0.632
t_detour (min)         -0.0074      0.005     -1.369      0.171      -0.018       0.003
t_congestion (min)     -0.0065      0.005     -1.373      0.170      -0.016       0.003
t_shift_late (min)     -0.0011      0.001     -0.732      0.464      -0.004       0.002
t_shift_early (min)    -0.0003      0.002     -0.194      0.846      -0.003       0.003
cost ($)               -0.0180      0.013     -1.364      0.173      -0.044       0.008
=======================================================================================
"""

In [26]:
mu1 = 1 + np.exp(-1*nested.params['Nest: canceling'])
print(f'Estimated Mu for canceling trip = {mu1:.4f}')
mu2 = 1 + np.exp(-1*nested.params['Nest: driving'])
print(f'Estimated Mu for driving = {mu2:.4f}')

Estimated Mu for canceling trip = 3.2496
Estimated Mu for driving = 3.5709
